Test Cristian Magallanes: 

In this exercise I will prepare and  aggregate the three types of data and calculate fields needed to join and tag according to agency code and destination code. For this case I'll prioricize First\-contact attribution of campaings and measure the times adn days these iterations take



In [ ]:
# Imports 

import pandas as pd
import datetime
import numpy as np
from IPython.display import Image

In [ ]:

# Define Gobal files parameters for read CSV

delimiter = ';'

encoding = 'latin-1'

file_path = 'data/'


In [ ]:
#Dictionary of destinations

dic_destinations = { 'varadero':'VAR', 'rivera maya':'RIV' ,  'punta cana':'PUN',  'Albânia':'ALB', 'menorca':'MEN',  'tenerife':'TFS',  'Maiorca':'PMI', 'gran canaria':'LPA', 'lanzarote': 'ACE',  'mallorca':'PMI'
 }

In [ ]:
# Define Bookings file atributes

# Define the file name
booking_file_name = 'TRADING_BOOKINGS.csv'

parse_dates = ['DAY_BW']

# Define the data types for each column
website_data_types = {'BOOKING_ID':str, 'BRAND':str, 'DAY_BW':str, 'DESTINATION_CODE':str, 'SUBPRODUCT_TYPE':str, 'DAY_TP':str, 'DAY_TP_END':str, 'TRAVEL_ORIGIN':str, 'DESTINATION_MARKET':str, 'PAXES':str, 'ADULTS':str, 'CHILDS':str, 'INFANTS':str, 'DAY_RETURN_FLIGHT':str, 'AGENCY_CODE':str, 'PVD':float, 'TTV':float, 'SALES_CHANNEL':str}

In [ ]:
# Read the CSV file
df_Bookings = pd.read_csv(file_path+booking_file_name ,  header='infer',  thousands='.',  decimal=',',  dtype=website_data_types, parse_dates=parse_dates, delimiter=delimiter, encoding=encoding)

In [ ]:
# Convert date fromat
df_Bookings['DAY_BW'] = pd.to_datetime(df_Bookings['DAY_BW'], format= "%d/%m/%Y %H:%M" )

# rename columns
df_Bookings.rename(columns={'BOOKING_ID': 'booking_id','DAY_BW': 'booking_time', 'TRAVEL_ORIGIN' : 'from', 'DESTINATION_CODE' : 'destination_code', 'BRAND': 'brand', 'SALES_CHANNEL': 'channel',  'AGENCY_CODE': 'agency_code', 'SUBPRODUCT_TYPE': 'subproduct',  'TTV': 'ttv'}, inplace=True)


In [ ]:
# Group data

df_Bookings = df_Bookings.groupby(['booking_id', 'booking_time',  'from', 'destination_code', 'agency_code', 'brand', 'channel']).agg(    ttv= ('ttv' , 'sum')
                                                                   ).reset_index()


In [ ]:
# Define Website file atributes

# Define the file name
website_file_name = 'WEBSITE_TRACKING.csv'

# Define the data types for each column
website_data_types = {'WEB_USER':str, 'EMAIL':str, 'TIME':int, 'CHARSET':str, 'LANGUAGE':str, 'SCREEN':str, 'VIEWPORT':str, 'TITTLE': str, 'DESCRIPTION': str, 'URL': str, 'DESTINATION_CODE':str, 'USER_AGENT':str, 'BROWSER':str, 'OS':str}

In [ ]:
# Read the CSV file
df_Website = pd.read_csv(file_path+website_file_name ,  header='infer',  thousands='.',  dtype=website_data_types, delimiter=delimiter, encoding=encoding)

In [ ]:
# Convert date fromat
df_Website['TIME'] = pd.to_datetime(df_Website['TIME'], unit='s')


# rename columns
df_Website.rename(columns={'EMAIL': 'email', 'TIME': 'time', 'TITTLE' : 'title', 'DESTINATION_CODE': 'destination_code'}, inplace=True)

# delete records without email 
df_Website = df_Website.dropna(subset=['email'])

# trim spaces
df_Website['email'] = df_Website['email'].str.strip()

In [ ]:
# Group data

df_Website = df_Website.groupby(['email', 'title', 'destination_code']).agg(    time= ('time' , 'min'),
                                                                                reach_website= ('email', 'size')
                                                                            ).reset_index()
df_Website['reach_website'] =df_Website['reach_website'].astype(int)


In [ ]:
# Define Email file atributes

# Define the file name
email_file_name = 'EMAIL_TRACKING.csv'

# Define the data types for each column
email_data_types = {'agency_code':str,'email':str, 'time':int, 'email_subject':str,'event_name':str}

In [ ]:
# Read the CSV file
df_Email = pd.read_csv(file_path+email_file_name ,  header='infer',  dtype=email_data_types, delimiter=delimiter, encoding=encoding)


In [ ]:
# Convert date format
df_Email['time'] = pd.to_datetime(df_Email['time'], unit='s')



In [ ]:
# Group data

df_Email['dropped'] = np.where(df_Email['event_name'] == 'email_dropped', 1, 0)
df_Email['delivered'] = np.where(df_Email['event_name'] == 'email_delivered', 1, 0)
df_Email['Open'] = np.where(df_Email['event_name'] == 'email_open', 1, 0)
df_Email['blocked'] = np.where(df_Email['event_name'] == 'email_blocked', 1, 0)
df_Email['click'] = np.where(df_Email['event_name'] == 'email_click', 1, 0)
df_Email['bounce'] = np.where(df_Email['event_name'] == 'email_bounce', 1, 0)
df_Email['unsubscribe'] = np.where(df_Email['event_name'] == 'email_unsubscribe', 1, 0)
df_Email['reach_email'] = np.where((df_Email['Open'] > 0) | (df_Email['click'] > 0), 1, 0)
df_Email = df_Email.groupby(['agency_code','email', 'email_subject' ]).agg({    'time': 'min',
                                                                                'delivered': 'sum', 
                                                                                'dropped': 'sum',
                                                                                'Open': 'sum', 
                                                                                'blocked': 'sum', 
                                                                                'click': 'sum', 
                                                                                'bounce': 'sum', 
                                                                                'unsubscribe': 'sum',
                                                                                'reach_email':'sum'
                                                                            }).reset_index()

# convert to integers

df_Email['delivered'] =df_Email['delivered'].astype(int) 
df_Email['dropped'] =df_Email['dropped'].astype(int) 
df_Email['Open'] =df_Email['Open'].astype(int) 
df_Email['blocked'] =df_Email['blocked'].astype(int) 
df_Email['click'] =df_Email['click'].astype(int) 
df_Email['bounce'] =df_Email['bounce'].astype(int) 
df_Email['unsubscribe'] =df_Email['unsubscribe'].astype(int) 
df_Email['reach_email'] =df_Email['reach_email'].astype(int) 
df_Email['email'] = df_Email['email'].str.strip()



In [ ]:
 # create agency code column on Website

df_Website['agency_code'] = df_Website['email'].map(df_Email.set_index('email')['agency_code'].to_dict())

In [ ]:
# Match destinations on Email

# switch dictionary
def look_destinations(x):
    for key, value in dic_destinations.items():
        if key in x:
            return value 
    return None  


df_Email['destination_code'] = df_Email['email_subject'].str.lower().apply(look_destinations)



In [ ]:
# Sample of data
df_Email.head()


In [ ]:
# Sample of data
df_Website.head()

In [ ]:
# Sample of data
df_Bookings.head()

In [ ]:
Image("images/agregate.png")


Agregate model with 3 enities where we can analyze campaings eficiency and client journey

In [ ]:
# union both Web and email activity
df_client_journey = [df_Website, df_Email]
df_client_journey = pd.concat(df_client_journey)


# calculate total  reach adding emails open and click and websites opened

df_client_journey['reach'] = df_client_journey['reach_website'].fillna(0) + df_client_journey['reach_email'].fillna(0)


In [ ]:
# Group data jorney

df_client_journey = df_client_journey.groupby(['agency_code', 'destination_code']).agg( {  
                                                                                'time': 'min',
                                                                                'reach':'sum'
                                                                            }
                                                                            ).reset_index()


df_client_journey['reach'] =df_client_journey['reach'].astype(int)

In [ ]:
# Join activity with bookings

df_client_journey = pd.merge(df_client_journey, df_Bookings, on=['agency_code', 'destination_code'], how='left')

df_client_journey = df_client_journey[['agency_code', 'destination_code', 'time', 'booking_time', 'brand', 'channel','from', 'reach', 'ttv']]



In [ ]:
# add meetric succesfull booking metrics where book was made after first atempt and ttv is not null


df_client_journey_book= df_client_journey.query ('time < booking_time and  ttv >= 0 and reach>0' ).copy()

df_client_journey['book'] = False

# add succesfull booking tag 

df_client_journey_book['book'] = True

# Add days between first interaction and booking
df_client_journey_book['dif_days'] =   (df_client_journey_book['booking_time'] - df_client_journey['time']).dt.days


df_client_journey['book'] = df_client_journey_book['book'] 

df_client_journey['dif_days'] = df_client_journey_book['dif_days'] 


df_client_journey["book"].fillna( False, inplace = True)






In [ ]:
# Add destination description

dic_destCodes = {v: k for k, v in dic_destinations.items()}

df_client_journey['destination_code'] = df_client_journey['destination_code'].map(dic_destCodes)


In [ ]:
df_client_journey.head()

In [ ]:
# Save csv for Analitics

df_client_journey.to_csv('data/data_analitic.csv', index=False)

The following imnage show screenshot of  Qlik View using data/data_analitic.csv

In [ ]:
Image("images/report.png")